In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import seaborn as sns
from datetime import datetime as dt
from datetime import timedelta as td
from scipy.integrate import simpson, cumtrapz

import DEBpython.pet as pet
import DEBpython.models as models
from DEBpython.visualizer import tex_par_symbols, tex_par_units, par_descriptions_en
import DEBpython.composition as composition
import DEBpython.formulae as formulae
import DEBpython.efficiency as efficiency

from DEBtoolPyIF.data_sources import *
from DEBtoolPyIF.notebook_visualizers import *

from run_multitier_estimation import create_tier_structure

sns.set_style('ticks')
%matplotlib inline

# Load data, parameters, errors, and auxiliary variables

In [3]:
multitier = create_tier_structure()
multitier.tiers['breed'].load_results()


In [4]:
multitier.tiers['breed'].pars_df

,p_Am,kap_X,p_M,v,kap,E_G,E_Hb,E_Hx,E_Hp,h_a,t_0,del_M,p_Am_f,E_Hx_f,E_Hp_f
tier_sample,,,,,,,,,,,,,,,
male,4312.79502,0.116091,106.035823,0.049424,0.926367,7835.884322,7.208394e+06,3.925416e+07,7.223944e+07,7.161304e-08,88.050549,0.221322,3806.691182,3.701081e+07,5.078369e+07
